<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [5]:
%pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [12]:
%pip install inference tensorflow[and-cuda] tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.5/907.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 940.0/940.0 kB 51.3 MB/s eta 0:

## Importing Libraries

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from tensorflow.keras import mixed_precision
import yaml
import os
from PIL import Image
from roboflow import Roboflow
from inference import get_model
import supervision as sv
import cv2
import numpy as np
from google.colab import userdata

## Setting up Environment

In [ ]:
api_key = userdata.get('ROBOFLOW_API_KEY')

In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['NO_ALBUMENTATIONS_UPDATE '] = '1'


In [7]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices())


TensorFlow version: 2.17.1
GPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [23]:
# List available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
# Enable memory growth for all GPUs
  for gpu in physical_devices:
      tf.config.experimental.set_memory_growth(gpu, True)

  # Use all available GPUs
  tf.config.set_visible_devices(physical_devices, 'GPU')

  # Enable XLA (Accelerated Linear Algebra) optimization
  tf.config.optimizer.set_jit(True)

  tf.config.set_visible_devices(physical_devices[0], 'GPU')

  # Enable mixed precision training

  policy = mixed_precision.Policy('mixed_float16')
  mixed_precision.set_global_policy(policy)

  print("GPU optimization settings applied successfully")
else:
    print("No GPU available. Running on CPU.")

Num GPUs Available:  1
GPU optimization settings applied successfully


## Loading Datasets

We're implementing various datasets from roboflow and our own custom images dataset for maximum training data

In [16]:

# Download both datasets
rf = Roboflow(api_key=api_key)

# Dataset 1

project1 = rf.workspace("yassine-pzpt7").project("sun-tracking-photovoltaic-panel")
version1 = project1.version(2)
dataset1 = version1.download("yolov8")

# Dataset 2
project2 = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
dataset2 = project2.version(2).download("yolov8")

# Dataset 3
project3 = rf.workspace("rik-tjduw").project("sun-tracking-555mn")
version3 = project3.version(4)
dataset3 = version3.download("yolov8")

# Dataset 4
project4 = rf.workspace("stardetect").project("solar-re1fe")
version4 = project4.version(1)
dataset4 = version4.download("yolov8")

# Dataset 5
project5 = rf.workspace("fruitdetection-ulcz9").project("sun_detection-hl04q")
version5 = project5.version(1)
dataset5 = version5.download("yolov8")

# Prepare dataset paths
dataset1_path = dataset1.location
dataset2_path = dataset2.location
dataset3_path = dataset3.location
dataset4_path = dataset4.location
dataset5_path = dataset5.location

def resize_images(directory, target_resolution=(640, 640)):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_path = os.path.join(root, file)
                try:
                    with Image.open(image_path) as img:
                        img_resized = img.resize(target_resolution, Image.LANCZOS)
                        img_resized.save(image_path)
                except Exception as e:
                    print(f"Error resizing image {image_path}: {e}")

# Function to count images in a directory
def count_images(directory):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    return len([f for f in os.listdir(directory) if f.lower().endswith(image_extensions)])

# Function to combine datasets

def combine_datasets(*dataset_paths, combined_path):
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(combined_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(combined_path, split, 'labels'), exist_ok=True)

        for dataset_path in dataset_paths:
            images_src = os.path.join(dataset_path, split, 'images')
            labels_src = os.path.join(dataset_path, split, 'labels')

            images_dst = os.path.join(combined_path, split, 'images')
            labels_dst = os.path.join(combined_path, split, 'labels')

            if os.path.exists(images_src):
                for file in os.listdir(images_src):
                    shutil.copy2(os.path.join(images_src, file), images_dst)
            if os.path.exists(labels_src):
                for file in os.listdir(labels_src):
                    shutil.copy2(os.path.join(labels_src, file), labels_dst)


# Combine datasets
combined_path = './combined_dataset'
combine_datasets(dataset1_path, dataset2_path, dataset3_path,dataset4_path, dataset5_path, combined_path=combined_path)

# Resize images in the combined dataset
for split in ['train', 'valid', 'test']:
    images_dir = os.path.join(combined_path, split, 'images')
    resize_images(images_dir)

print("Images resized to target resolution.")


# Count images in each dataset
train_count = len(os.listdir(os.path.join(combined_path, 'train', 'images')))
valid_count = len(os.listdir(os.path.join(combined_path, 'valid', 'images')))
test_count = len(os.listdir(os.path.join(combined_path, 'test', 'images')))
num_classes = len(set([f.split('_')[0] for f in os.listdir(os.path.join(combined_path, 'train', 'labels'))]))

print(f"Number of images in training set: {train_count}")
print(f"Number of images in validation set: {valid_count}")
print(f"Number of images in test set: {test_count}")
print(f"Total number of images: {train_count + valid_count + test_count}")

# Create YAML file for YOLOv5
yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': len(os.listdir(os.path.join(combined_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(combined_path, 'train', 'labels'))))],
    'image_counts': {
        'train': train_count,
        'val': valid_count,
        'test': test_count
    }
}


yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': num_classes,
    'names': [f'class_{i}' for i in range(num_classes)],
}

yaml_path = os.path.join(combined_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Dataset YAML file created at: {yaml_path}")


loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Images resized to target resolution.
Number of images in training set: 3597
Number of images in validation set: 633
Number of images in test set: 443
Total number of images: 4673
Dataset YAML file created at: ./combined_dataset/dataset.yaml


## Setting Up Models


### 2. YoloV5 (LibRT)

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
from ultralytics import YOLO

yolo_model = YOLO('yolov8l.pt')

full_yaml_path = os.path.abspath(yaml_path)
yolo_results = yolo_model.train(
    data=full_yaml_path,
    epochs=500,
    imgsz=640,
    lr0=0.001,
    lrf=0.0001,
    batch=-1,
    device=[0],
    augment=True,
    cos_lr=True,
    amp=True,
    resume=False,
    patience=300,
    mixup=0.2,
    mosaic=1.0,
    close_mosaic=10
)



Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/combined_dataset/dataset.yaml, epochs=500, time=None, patience=300, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /content/combined_dataset/train/labels.cache... 3597 images, 218 backgrounds, 0 corrupt: 100%|██████████| 3597/3597 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 579, len(boxes) = 3384. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA L4) 22.17G total, 1.91G reserved, 1.56G allocated, 18.70G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    43895064       166.9         3.678          37.1         21.04        (1, 3, 640, 640)                    list
    43895064       333.8         4.255         38.03         24.97        (2, 3, 640, 640)                    list
    43895064       667.5         5.478         48.17          33.7        (4, 3, 640, 640)                    list
    43895064        1335         7.841         63.77         75.39        (8, 3, 640, 640)                    list
    43895064        2670        12.562         154.8           170       (16, 3, 640, 640)                    list
    43895064        5340        22.398         350.9         370.8       (32, 3, 640, 640)                    list
CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 2.88 MiB is free. Process 69173 has 22.15 GiB memory in use. Of the allocated memory 21.71 GiB is allocated by PyTorch, and 173.18 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is

train: Scanning /content/combined_dataset/train/labels.cache... 3597 images, 218 backgrounds, 0 corrupt: 100%|██████████| 3597/3597 [00:00<?, ?it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     82/500      11.9G      1.068     0.8053      1.339         20        640: 100%|██████████| 225/225 [01:55<00:00,  1.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:09<00:00,  2.11it/s]

                   all        633        614      0.846      0.909      0.915      0.713



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     83/500      11.7G       1.09     0.8295      1.352         27        640:  76%|███████▌  | 170/225 [01:27<00:28,  1.95it/s]

In [ ]:
# Evaluate the model after training
metrics = yolo_model.val()
print(f"mAP50-95: {metrics.box.map}")


## Testing Models

In [ ]:
# tbd

## Saving Model

In [ ]:
yolo_model.save('sun_tracker_model.h5')


## Pretrained Model Implementation

Static Models from roboflow web


| Model | mAP50-95 | Precision | Recall | Images | Type |
|-------|----------|-----------|--------|--------|------|
| Our YOLO model | 73.5% | 89.2% | 91.7% | 633 | YOLOv8 (custom trained) |
| solar-re1fe/1 | 96.8% | 95.2% | 94.3% | 2684 | Roboflow 3.0 Object Detection (Fast) |
| sun-tracking-photovoltaic-panel/1 | 98.2% | 93.7% | 93.7% | 196 | Roboflow 2.0 Object Detection (Fast) |
| sun-tracking/3 | 92.5% | 94.7% | 91.8% | 1090 | YOLOv5 Model Upload |
| sun-tracking-555mn/4 | 97.7% | 93.0% | 89.2% | 923 | YOLOv8n Model Upload |



1. **sun-tracking-555mn/4**
   - Best overall performance with 97.7% mAP
   - YOLOv8n architecture, which is lightweight and suitable for microcontrollers
   - Recently trained (2024-03-05)
   - Balanced dataset size (923 images)

2. **solar-re1fe/1**
   - High performance (96.8% mAP, 95.2% precision, 94.3% recall)
   - Largest dataset (2684 images), potentially good for generalization
   - Roboflow 3.0 Object Detection (Fast) may be suitable for microcontrollers

3. **sun-tracking-photovoltaic-panel/1**
   - Highest mAP (98.2%)
   - Roboflow 2.0 Object Detection (Fast) may work on microcontrollers
   - Limited dataset (196 images) might affect generalization

4. **sun-tracking/3**
   - Lowest mAP (92.5%) among the options
   - YOLOv5 architecture, which is generally suitable for microcontrollers
   - Decent dataset size (1090 images)


In [2]:


# define the image url to use for inference
image_file = "test/1.jpg"
image = cv2.imread(image_file)

# load a pre-trained yolov8n model
model = get_model(model_id="sun-tracking-555mn/4", api_key=api_key)

# run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
results = model.infer(image)[0]

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(results)

# create supervision annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# display the image
sv.plot_image(annotated_image)

NameError: name 'cv2' is not defined

### Implementing Live Web Cam

#### Central Grid Vision

A central gride box which would determine the strongest position for the solar panel to position for maximum sunlight. The solar panel will follow and turn according to the distance between the central grid and the sun detected in camera

![image-1.png](https://github.com/somwrks/AI-Solar-Panel/blob/main/image-1.png?raw=true)

In [ ]:

def draw_central_box(frame, box_size=100):
    height, width = frame.shape[:2]
    center_x, center_y = width // 2, height // 2
    top_left = (center_x - box_size // 2, center_y - box_size // 2)
    bottom_right = (center_x + box_size // 2, center_y + box_size // 2)
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
    return top_left, bottom_right

def calculate_distance_to_box(sun_center, top_left, bottom_right):
    sun_x, sun_y = sun_center
    box_x_min, box_y_min = top_left
    box_x_max, box_y_max = bottom_right

    closest_x = max(box_x_min, min(sun_x, box_x_max))
    closest_y = max(box_y_min, min(sun_y, box_y_max))

    return sun_x - closest_x, sun_y - closest_y


Applying different image filters for detecting sun in various environments

In [ ]:
def apply_sun_filter(frame):
    # Convert to float32 for better precision in calculations
    img = cv2.convertScaleAbs(frame, alpha=1.2)

    # Convert to HSV for better control
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #  contrast enhancement
    hsv[:,:,2] = cv2.convertScaleAbs(hsv[:,:,2], alpha=-0.40)


    # # Convert back to BGR
    filtered = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return filtered

### Setting up pipeline

In [1]:
# Load the pre-trained model
model = get_model(model_id="sun-tracking-555mn/4", api_key=api_key)

# Create supervision annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply sun filter
    filtered_frame = apply_sun_filter(frame)

    # Draw central box
    top_left, bottom_right = draw_central_box(filtered_frame)

    # Run inference on filtered frame
    results = model.infer(filtered_frame)[0]
    detections = sv.Detections.from_inference(results)

    if len(detections) > 0:
        for bbox in detections.xyxy:
            sun_center = ((bbox[0] + bbox[2])//2, (bbox[1] + bbox[3])//2)
            dx, dy = calculate_distance_to_box(sun_center, top_left, bottom_right)
            cv2.putText(filtered_frame, f"dx: {dx}, dy: {dy}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print(f"dx: {dx}, dy: {dy}", (10, 30))

    # Annotate and display frame
    annotated_frame = bounding_box_annotator.annotate(scene=filtered_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

    cv2.imshow("Filtered Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'get_model' is not defined